In [1]:
!pip install openai==0.28.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [2]:
!pip install tiktoken==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.2 MB/s eta 0:00:00


In [3]:
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-spli

In [4]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [5]:
os.environ['OPENAI_API_KEY'] = ''

In [6]:
openai.api_key = ''

In [7]:
System_Prompt = """

Role:
You are an AI-powered customer support chatbot designed to assist customers of an Apple-affiliated product distribution company. Your primary role is to help customers inquire about and track their order status. Additionally, you provide responses to general questions about shipping, delivery times, and product specifications as they relate to orders.

Identity:
Your identity reflects a helpful, professional, and friendly tone, in line with Apple’s brand values. You should communicate with clarity, professionalism, and empathy, ensuring customers feel supported and informed.

Context:
Customers will reach out with questions primarily focused on order status, shipping details, and delivery updates. Each interaction should begin with a greeting and proceed by verifying the customer’s order details, typically using an order number or email. You have access to a real-time order tracking system and can provide updates on the shipment status, estimated delivery times, and general product information. If a customer has complex issues (e.g., changing an order or needing refund details), you should politely redirect them to a human support agent.

Capabilities:
You can retrieve order status, shipping details, and estimated delivery times based on customer input. Additionally:
You are capable of handling common queries regarding product specifications and limited order modifications.
You can escalate issues that are beyond your capabilities to a human agent, ensuring a smooth handover.
You provide error-handling responses when you cannot retrieve information, offering alternative solutions (e.g., verifying details or contacting support).

Environment:
This chatbot will operate across various customer support platforms, including the website, mobile app, and possibly social media channels. Customers may have varying levels of technical knowledge, so your responses should be accessible, providing clear instructions and optional “quick reply” buttons where possible.

Example:

Customer: "Hi, can you tell me where my order is?"

Chatbot Response:
“Hello! I’d be happy to help you with that. Could you please provide your order number or the email used for the purchase? This will allow me to check the status of your order.”

"""

In [8]:
struct = [{"role" : "system" , "content" : System_Prompt}]

In [9]:
dataframe = pd.read_csv('https://raw.githubusercontent.com/jaydiaz2012/AI_RAG_Dataset_live/refs/heads/main/shipment_orders_apple_products.csv')

In [10]:
dataframe.head()

,Order ID,Product Name,Model,Quantity,Order Date,Ship Date,Delivery Date,Origin Location,Destination Location,City,Shipping Status,Customer Name,Customer Contact,Carrier,Shipping Cost,Tracking Number
0,1001,iPhone,iPhone 15,20,2024-11-01,2024-11-02,2024-11-04,"Cupertino, CA","New York, NY",New York,In Transit,XYZ Retail,123-456-7890,FedEx,200,FX123456789
1,1002,MacBook,MacBook Air,15,2024-11-01,2024-11-02,2024-11-05,"Cupertino, CA","Chicago, IL",Chicago,Delivered,ABC Corp,987-654-3210,UPS,150,UPS987654321
2,1003,iPad,iPad Pro,10,2024-11-02,2024-11-03,2024-11-06,"Cupertino, CA","Miami, FL",Miami,Shipped,DEF Wholesale,555-111-2222,DHL,180,DHL555111222
3,1004,Apple Watch,Series 9,30,2024-11-02,2024-11-03,2024-11-06,"Cupertino, CA","Los Angeles, CA",Los Angeles,Preparing,GHI Distributors,444-333-2222,FedEx,160,FX444333222
4,1005,iPhone,iPhone SE,25,2024-11-03,2024-11-04,2024-11-07,"Cupertino, CA","Houston, TX",Houston,In Transit,JKL Electronics,666-555-4444,UPS,210,UPS666555444


In [11]:
dataframe['combined'] = dataframe.apply(lambda row : ' ' .join(row.values.astype(str)), axis = 1)

In [12]:
documents = dataframe['combined'].tolist()

In [13]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [14]:
embeddings_dim = len(embeddings[0])

In [15]:
embeddings_np = np.array(embeddings).astype('float32')

In [16]:
index = faiss.IndexFlatL2(embeddings_dim)

In [17]:
index.add(embeddings_np)

In [55]:
user_message = "Hi, I represent the ABC Corp, one of your customers. Can you tell me the shipping status of my order, MacBook Air?"

In [56]:
query_embedding = get_embedding(user_message, engine = "text-embedding-3-small")
query_embedding_np = np.array(query_embedding).astype('float32')

In [57]:
query_embedding_np

array([ 0.03370721, -0.01019509, -0.00970729, ..., -0.01493897,
        0.00337499,  0.02375602], dtype=float32)

In [59]:
query_embedding_np = query_embedding_np.reshape(1, -1)

In [60]:
_, indices = index.search(np.array(query_embedding_np), 2)

In [61]:
retrieved_docs = [documents [i] for i in indices[0]]

In [62]:
context = ' '.join(retrieved_docs)

In [63]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [64]:
print(structured_prompt)

Context:
1002 MacBook MacBook Air 15 2024-11-01 2024-11-02 2024-11-05 Cupertino, CA Chicago, IL Chicago Delivered ABC Corp 987-654-3210 UPS 150 UPS987654321 1013 MacBook MacBook Pro 18 2024-11-06 2024-11-07 2024-11-12 Cupertino, CA San Diego, CA San Diego Shipped HIJ Corp 123-123-1234 FedEx 210 FX123123123

Query:
Hi, I represent the ABC Corp, one of your customers. Can you tell me the shipping status of my order, MacBook Air?

Response:


In [65]:
print(user_message)

Hi, I represent the ABC Corp, one of your customers. Can you tell me the shipping status of my order, MacBook Air?


In [66]:
chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [67]:
print(response)

“Hello! Thank you for reaching out, and I appreciate your representation of ABC Corp. 

The shipping status for your order, **MacBook Air** (Order ID 1002), is as follows:

- **Status**: Delivered
- **Delivery Date**: November 5, 2024
- **Shipping Carrier**: UPS
- **Tracking Number**: UPS987654321

If you need any further assistance or have additional questions, feel free to ask!”
